This is an implementation for phase flip error correction.

This code is inspired by the following quantum computing tutorial (https://quantumcomputinguk.org/tutorials/quantum-error-correction-phase-flip-code-in-qiskit). Throughout the code, there will be comments describing the process and the circuit creation.

Due to the nature of the hardware of quantum computers, it is possible for errors such as unwanted bit and phase flips to occur on qubits. In classical computers, if we are suspecting corruption of a bit or many bits, we can save copies to secure it/them. On the other hand, in quantum computers when a qubit is in a superposition between |0> and |1> (|ψ>= α|0> + β|1>), then it is impossible to copy it, which is also known as the no-cloning theorem. 

In order to protect our qubit from potential errors, we can design a circuit that uses ancillary qubits and the property of entanglement. This circuit proves that it is possible to correct a potential unwanted phase flip, by inducing a phase flip on the original qubit q<sub>0</sub> and then showing that at the end of the circuit, this error is corrected for.

Here is the circuit overview:

1. Ancillary entanglement
2. Hadamard gate
3. Phase flip of original qubit (This is done intentionally in our case)
4. Hadamard gate
5. CNOT and Toffoli

Lets first import our required libraries and load our account (we will run this code on a simulator)

In [ ]:
from qiskit import QuantumRegister
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute,IBMQ
from qiskit.tools.monitor import job_monitor
from qiskit.extensions import Initialize

provider = IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')

backend = provider.get_backend('ibmq_qasm_simulator')

Initialize our circuit. All qubits are initialized in the state |0>. After this circuit, I will also try to initialize qubit q<sub>0</sub> with the |1> state and check if we measure |1> at the end

In [ ]:
q = QuantumRegister(3,'q')
c = ClassicalRegister(1,'c')

circuit = QuantumCircuit(q,c)

1. Ancillary entanglement
-------------------------

In this first part of the circuit, we will use controlled NOT gates to induce entanglement between the ancillary qubits and the original qubit. I found this slide from John Preskill be usefull in understanding this concept:

<img src="Preskill2.png" alt="Output5">

In [ ]:
circuit.cx(q[0],q[1])
circuit.cx(q[0],q[2])

circuit.draw()

Output:

<img src="Output5.png" alt="Output5">

2. Hadamart gates
-----------------

Now that our three qubits are entangled, we apply the hadamart gate on all of them. The reason we apply Hadamard gates before and after our Z gate is that we are practically transforming our circuit into a bit flip error correction circuit. I tried doing the math for a one qubit HZH and I got that HZH = X. Thus, our phase flip error correction circuit turns into a bit flip error correction circuit, while we still have our Z gate to simulate a potential phase flip. An intuition for this is that its not practically possible to observe a phase flip, whereas we can observe a bit flip. Here is the first application of H gates:

In [ ]:
circuit.h(q[0])
circuit.h(q[1])
circuit.h(q[2]) 

circuit.draw()

Output:

<img src="Output6.png" alt="Output6">

3. Phase flip on the qubit q<sub>0</sub>
---------------------------------------

This step is done to purposely induce a phase flip on qubit q<sub>0</sub>, in order to show that at the end of the circuit, we will get the original qubit state back. Its made to replace the potential error that can occur in pratice. 

In [ ]:
circuit.z(q[0])

circuit.draw()

Output:

<img src="Output7.png" alt="Output7">

4. Hadamart gates
-----------------

This is the second application of our H gates:

In [ ]:
circuit.h(q[0])
circuit.h(q[1])
circuit.h(q[2])

circuit.draw()

Output:

<img src="Output8.png" alt="Output8">

5. CNOTs and Toffoli
--------------------

The last step is to perform the CNOT gates again, followed by a Toffoli gate with the controllers being the ancillary qubits and the target qubit being q<sub>0</sub>. This step restores q<sub>0</sub> into its original state (before the phase flip change). After the Toffoli gate, we measure qubit q<sub>0</sub> and indeed we recover its original state.

In [ ]:
circuit.cx(q[0],q[1])
circuit.cx(q[0],q[2])
circuit.ccx(q[2],q[1],q[0])
circuit.measure(q[0],c[0])
circuit.draw()

Output:

<img src="Output9.png" alt="Output9">

We run the circuit on a simulator and we check the result.

In [ ]:
job = execute(circuit, backend, shots=1000)
job_monitor(job)
counts = job.result().get_counts()

print(counts)

Output:

{'0': 1000}

We repeated this experiment 1000 times and we measured 0 everytime. Lets now try to initialize our qubit q<sub>0</sub> to be at state |1> and see if we measure 1 at the end  

**Extra - Measurement for state |1>**

In [ ]:
q = QuantumRegister(3,'q')
c = ClassicalRegister(1,'c')

circuit = QuantumCircuit(q,c)

circuit.x(q[0]) #  By adding this X gate, we are flipping q[0] from 0 to 1

circuit.cx(q[0],q[1])
circuit.cx(q[0],q[2])

circuit.h(q[0])
circuit.h(q[1])
circuit.h(q[2]) 

circuit.z(q[0])

circuit.h(q[0])
circuit.h(q[1])
circuit.h(q[2])

circuit.cx(q[0],q[1])
circuit.cx(q[0],q[2])
circuit.ccx(q[2],q[1],q[0])
circuit.measure(q[0],c[0])
circuit.draw()

Output:

<img src="Output10.png" alt="Output10">

This circuit is the same as the previous one, except here we added an X gate for qubit q<sub>0</sub>, which will flip it from state |0> to state |1>

We execute

In [ ]:
job = execute(circuit, backend, shots=1000)
job_monitor(job)
counts = job.result().get_counts()

print(counts)

Output:

{'1': 1000}

It seems like we are now measuring 1 at the end of our circuit instead of 0, as we expected. This experiment was done as in the original experiment, the CNOT gates dont change anything to the state of q<sub>1</sub> and q<sub>2</sub>.